In [9]:
from agents.mcp import MCPServerStdio
from agent_tools import send_email
import os

In [2]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [3]:
send_email(body="This is an email test", subject="My Test email")

UnauthorizedError: HTTP Error 401: Unauthorized

### Running node based mcp server tools

In [5]:
playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"]}
# playwright_params = {"command": "npx", "args": []}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


NameError: name 'MCPServerStdio' is not defined

In [5]:

file_storage_path = os.path.abspath(os.path.join(os.getcwd(), "file_storage"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", file_storage_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', title=None, description="Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Only works within allowed directories.", inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_multiple_files', title=None, description="Read the contents of multiple files simultaneously. This is

### Presenting said tools to an LLM to use and test

In [5]:
from openai_sdk_resume_assistant.client import AzureAIClient
from openai import AsyncAzureOpenAI
from dotenv import load_dotenv
from agents import Runner, set_default_openai_api, set_default_openai_client, set_tracing_disabled, Agent
import os
from agents.mcp import MCPServerStdio

In [2]:
load_dotenv(override=True)

client = AzureAIClient()
openai_client = AsyncAzureOpenAI(
    azure_endpoint = client.azure_endpoint,
    api_version = client.api_version,
    azure_ad_token_provider = client.token_provider
)

/usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize?client_id=04b07795-8ddb-461a-bbee-02f9e1bf7b46&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8400&scope=https%3A%2F%2Fcognitiveservices.azure.com%2F.default+offline_a

In [3]:
# Set default client and configs
set_default_openai_client(openai_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

In [6]:
# Create mcp servers with the params
playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"]}
file_storage_path = os.path.abspath(os.path.join(os.getcwd(), "file_storage"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", file_storage_path]}


params_list = [playwright_params, files_params]
tool_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in params_list]

In [7]:
tool_agent_instructions= "You are a tool agent that can use tools for browsing the internet and you can also have access to my file system. You are highly capable to accomplish your tasks independently. This includes accepting all cookies and clicking 'not now' as appropriate to get the content you need. If one website isn't fruitful, try another. Be persistent until you are able to solve your assignment, trying different options and sites are needed."

In [8]:
tool_agent = Agent(
    name="ToolAgent",
    instructions=tool_agent_instructions,
    model="gpt-4o",
    mcp_servers = tool_mcp_servers
)

In [20]:
user_input = "Find some very relevant information about the company Anthropic and summarize to the file anthropic2.md"
user_input2 = "Find some relevant information about Anthropic the company and summarize it to the file called Anthropic_update.md"

In [21]:
# Connect to the servers 
for server in tool_mcp_servers:
    await server.connect()

response = await Runner.run(tool_agent, input = user_input2)

In [22]:
print("Response:", response.final_output)

Response: I have successfully written the information about Anthropic into the file `Anthropic_update.md`. You can find it in the directory `/mnt/c/Users/edan/repos/personal_projects/AI/Agentic_AI_Projects/openai_sdk_resume_assistant/src/openai_sdk_resume_assistant/file_storage/`.


In [13]:
# Connect to the servers 
for server in tool_mcp_servers:
    await server.connect()
    
result = await Runner.run(tool_agent, "Find a good shepards pie recipe and summarize it professionally in a markdown called shepards_pie.md")

In [22]:
from IPython.display import Markdown, display
display(Markdown(result.final_output))

I have successfully created the markdown file `shepards_pie.md` with the Shepherd's Pie recipe. You can find the file in the following directory:

`/mnt/c/Users/edan/repos/personal_projects/AI/Agentic_AI_Projects/openai_sdk_resume_assistant/src/openai_sdk_resume_assistant/file_storage/shepards_pie.md`